In [42]:
#参考: https://www.kaggle.com/c/word2vec-nlp-tutorial/overview/part-1-for-beginners-bag-of-words
#本篇是使用sklearn中linear_model基于bagOfWords的文本分类代码
#todo:等以后有时间实现基于numpy的逻辑回归

In [43]:
import config as cfg
import pandas as pd
from os.path import join
import re
import nltk
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
#nltk.download()
from nltk.corpus import stopwords # Import the stop word list
DATA_DIR = cfg.DATA_PATH

In [44]:
def load_data():
    train_df = pd.read_csv(join(DATA_DIR, 'train.tsv'), delimiter='\t')
    X_train,X_test, y_train, y_test =train_test_split(train_df['Phrase'],train_df['Sentiment'],test_size=0.3, random_state=0)
    return X_train,X_test, y_train, y_test
X_train,X_test, y_train, y_test = load_data()
print(len(X_train))
print(len(X_test))
print(len(stopwords.words("english") ))

109242
46818
179


In [45]:
#文本预处理
def review_to_words( raw_review ):
    #除掉非字母字符
    letters_only = re.sub("[^a-zA-Z]", " ", raw_review) 
    
    #小写化
    words = letters_only.lower().split()                             
    
    #在set中搜索比在list中快很多
    stops = set(stopwords.words("english"))                  

    #去停用词
    meaningful_words = [w for w in words if not w in stops]   
    
    return( " ".join( meaningful_words ))
X_train = [review_to_words(review) for review in tqdm(X_train)]
X_test = [review_to_words(review) for review in tqdm(X_test)]

100%|██████████████████████████████████████████████████████████████████████████| 46818/46818 [00:24<00:00, 1914.18it/s]


In [46]:
#统计词袋
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 10000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
X_train_features = vectorizer.fit_transform(X_train)
X_train_features = X_train_features.toarray()

X_test_features = vectorizer.fit_transform(X_test)
X_test_features = X_test_features.toarray()

In [47]:
from sklearn import linear_model

# 训练逻辑回归模型
logreg = linear_model.LogisticRegression()
logreg.fit(X_train_features, y_train)
 
# 预测
prepro = logreg.predict_proba(X_test_features)
acc = logreg.score(X_test_features,y_test)
print(acc)

C:\ProgramData\Anaconda3\envs\torch_env\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\envs\torch_env\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.4831047887564612


In [48]:
np.sum(prepro.argmax(axis=1)==y_test)/len(y_test)

0.4831047887564612